In [0]:
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql import Row
from delta.tables import DeltaTable

In [0]:
def updateFileLookup(table_name, status):
    fileLookup = spark.read.table("inventory_project.metadata.filelookup")
    # Filter only the specific table row
    df = fileLookup.filter(col("table_name") == table_name) \
                   .withColumn("lastRunStatus", lit(status)) \
                   .withColumn("lastUpdatetime", current_timestamp())
    
    target_table = DeltaTable.forName(spark, "inventory_project.metadata.filelookup")

    # Perform merge (upsert)
    target_table.alias("t") \
        .merge(
            df.alias("s"),
            "t.table_name = s.table_name"
        ) \
        .whenMatchedUpdate(set={              
            "lastRunStatus": "s.lastRunStatus",
            "lastUpdatetime": "s.lastUpdatetime"
        }) \
        .whenNotMatchedInsertAll() \
        .execute()
